## Task -1 Exploratory Data Analysis

In [1]:
"""
Installing Spark with its dependencies
Installing Spark
Install Dependencies:

Java 8
Apache Spark with hadoop and
Findspark (used to locate the spark in the system)
"""

#these operations could be done on Google colab or needed to run under linuxOS.
 

#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
#!tar xf spark-3.1.1-bin-hadoop3.2.tgz
#!pip install -q findspark

'\nInstalling Spark with its dependencies\nInstalling Spark\nInstall Dependencies:\n\nJava 8\nApache Spark with hadoop and\nFindspark (used to locate the spark in the system)\n'

In [2]:
#Set Environment Variables:

import os

current_directory = os.getcwd()


os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = os.path.join(current_directory,"spark-3.1.1-bin-hadoop3.2")

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

24/01/23 12:01:19 WARN Utils: Your hostname, gormezoglu-linux resolves to a loopback address: 127.0.1.1; using 192.168.1.86 instead (on interface wlp3s0)
24/01/23 12:01:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/23 12:01:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
#will be used sqlite3 to be able to reach .db file

import sqlite3

con = sqlite3.connect('Datasets/movielens-small.db')
cur = con.cursor()

In [5]:
#For SQLite JDBC driver, it can be downloaded via:

!curl -O https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.34.0/sqlite-jdbc-3.34.0.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7125k  100 7125k    0     0  2074k      0  0:00:03  0:00:03 --:--:-- 2074k


In [6]:
# Write a SQL query to create a dataframe with including userid, movieid, genre and rating

import csv
with open("movielens.csv", "w") as csvFile:
    fieldnames = ['userId', 'movieId', 'genre', 'rating']
    writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
    writer.writeheader()

    for row in cur.execute("""SELECT ratings.userId, movies.movieId, genres, rating 
                              FROM (((movies 
                                      INNER JOIN links ON movies.movieid = links.movieid) 
                                      LEFT JOIN ratings ON movies.movieId = ratings.movieId) 
                                      LEFT JOIN tags ON movies.movieid = tags.movieid)"""):
 
        userId = row[0]
        movieId = row[1]
        genre = row[2]
        rating = row[3]

        writer.writerow({'userId': userId, 
                        'movieId':movieId,
                        'genre': genre,
                        'rating':rating}
        )

movielens_small_df = spark.read.csv("movielens.csv", header=True)

print("number of rows of our dataframe:", movielens_small_df.count())
print("\n")

movielens_small_df

number of rows of our dataframe: 249641




userId,movieId,genre,rating
7,1,Adventure|Animati...,5.0
7,1,Adventure|Animati...,5.0
7,1,Adventure|Animati...,5.0
10,1,Adventure|Animati...,4.0
10,1,Adventure|Animati...,4.0
10,1,Adventure|Animati...,4.0
13,1,Adventure|Animati...,4.5
13,1,Adventure|Animati...,4.5
13,1,Adventure|Animati...,4.5
16,1,Adventure|Animati...,5.0


In [7]:
# Count ratings for each movie, and list top 5 movies with the highest value

movie_rating_count = []

for row in cur.execute("""SELECT movieid, title,COUNT(rating) 
                          FROM (SELECT ratings.userId, movies.movieId, title,genres, rating 
                                FROM (((movies 
                                        INNER JOIN links ON movies.movieid = links.movieid)
                                        LEFT JOIN ratings ON movies.movieId = ratings.movieId)
                                        LEFT JOIN tags ON movies.movieid = tags.movieid))                                                    
                          GROUP BY movieid
                          ORDER BY count(rating) DESC
                          LIMIT 5
                        """):
  movie_rating_count.append(row)

schema = ["movieId", "title", "rating count"]
 
# calling function to create dataframe
df = spark.createDataFrame(movie_rating_count, schema)

df.show(truncate=False)
  

+-------+-------------------------+------------+
|movieId|title                    |rating count|
+-------+-------------------------+------------+
|318    |Shawshank Redemption, The|5904        |
|2571   |Matrix, The              |5300        |
|296    |Pulp Fiction             |4578        |
|2762   |Sixth Sense, The         |4080        |
|47     |Seven (a.k.a. Se7en)     |4047        |
+-------+-------------------------+------------+



In [8]:
# Find and list top 5 most rated genres

rated_genres = []

for row in cur.execute("""SELECT genres, COUNT(rating) 
                          FROM (SELECT ratings.userId, movies.movieId, genres, rating 
                                FROM (((movies 
                                        INNER JOIN links ON movies.movieid = links.movieid) 
                                        LEFT JOIN ratings ON movies.movieId = ratings.movieId) 
                                        LEFT JOIN tags ON movies.movieid = tags.movieid))
                          GROUP BY genres
                          ORDER BY COUNT(rating) DESC
                          LIMIT 5
                        """):
  rated_genres.append(row)

schema = ["genres", "rating count"]
 
# calling function to create dataframe
df = spark.createDataFrame(rated_genres, schema)

df.show(truncate=False)


+--------------------------------+------------+
|genres                          |rating count|
+--------------------------------+------------+
|Comedy                          |10162       |
|Crime|Drama                     |10107       |
|Comedy|Drama                    |7949        |
|Drama                           |7846        |
|Action|Adventure|Sci-Fi|Thriller|7182        |
+--------------------------------+------------+



In [9]:
#Find and list top 5 most rated tags

rated_tags = []

for row in cur.execute("""SELECT tag, COUNT(rating) 
                          FROM (SELECT ratings.userId, movies.movieId, genres, tag, rating 
                                FROM (((movies 
                                        INNER JOIN links ON movies.movieid = links.movieid) 
                                        LEFT JOIN ratings ON movies.movieId = ratings.movieId) 
                                        LEFT JOIN tags ON movies.movieid = tags.movieid))
                          WHERE tag is NOT NULL
                          GROUP BY tag
                          ORDER BY COUNT(rating) DESC
                          LIMIT 5
                        """):
  rated_tags.append(row)

schema = ["tags", "tags count"]
 
# calling function to create dataframe
df = spark.createDataFrame(rated_tags, schema)

df.show(truncate=False)



+------------+----------+
|tags        |tags count|
+------------+----------+
|drama       |3542      |
|sci-fi      |3035      |
|twist ending|2998      |
|psychology  |2672      |
|crime       |2570      |
+------------+----------+



In [10]:
# By using timestamp from ratings table, provide top 5 most frequent users within a week

weekly_activity = []

for row in cur.execute("""SELECT userid, strftime('%Y-%W', datetime(timestamp, 'unixepoch')) AS week_year, COUNT(strftime('%Y-%W', datetime(timestamp, 'unixepoch'))) AS weekly_activity 
                          FROM ratings
                          GROUP BY userid, week_year 
                          ORDER BY weekly_activity DESC
                          LIMIT 5
                        """):
  weekly_activity.append(row)

schema = ["userId", "week of the year", "weekly activity of user"]
 
# calling function to create dataframe
df = spark.createDataFrame(weekly_activity, schema)

df.show(truncate=False)



+------+----------------+-----------------------+
|userId|week of the year|weekly activity of user|
+------+----------------+-----------------------+
|384   |2008-42         |1107                   |
|176   |2015-03         |972                    |
|614   |2014-27         |830                    |
|330   |2008-33         |711                    |
|202   |2003-50         |676                    |
+------+----------------+-----------------------+



In [11]:
# Calculate average ratings for each genre, and plot average ratings of top 10 genres with descending order

genres_avgrating=[]

for row in cur.execute("""SELECT genres, round(avg(rating), 2) avg_rating 
                          FROM (SELECT ratings.userId, movies.movieId, genres, rating 
                                FROM (((movies 
                                        INNER JOIN links ON movies.movieid = links.movieid) 
                                        LEFT JOIN ratings ON movies.movieId = ratings.movieId) 
                                        LEFT JOIN tags ON movies.movieid = tags.movieid))
                          GROUP BY genres
                          ORDER BY avg_rating DESC
                          LIMIT 10"""):
  genres_avgrating.append(row)

schema = ["genres", "avg_rating"]
 
# calling function to create dataframe
df = spark.createDataFrame(genres_avgrating, schema)

df.show(truncate=False)



+--------------------------------------------------------+----------+
|genres                                                  |avg_rating|
+--------------------------------------------------------+----------+
|Crime|Documentary|War                                   |5.0       |
|Animation|Documentary                                   |5.0       |
|Adventure|Fantasy|Mystery                               |5.0       |
|Action|Adventure|Animation|Comedy|Fantasy|Mystery|Sci-Fi|5.0       |
|Crime|Horror|Mystery                                    |4.75      |
|Drama|Horror|War                                        |4.5       |
|Comedy|Crime|Western                                    |4.5       |
|Children|Horror|Mystery|Thriller                        |4.5       |
|Animation|Comedy|Horror|IMAX                            |4.5       |
|Animation|Children|Drama|Fantasy|IMAX                   |4.5       |
+--------------------------------------------------------+----------+



## TASK 2 - Recommender Design

In [12]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [13]:
spark = SparkSession.builder.appName("movieRecommendation").getOrCreate() 

In [14]:
# using movie rating data to probide implicit feature using ALS(Alternate Least Squares)

movie_rating=[]

for row in cur.execute("""SELECT userId, movieId, rating 
                          FROM ratings
                                      """):
  movie_rating.append(row)

schema = ["userId", "movieId", "rating"]
 
# calling function to create dataframe
movie_rating_df = spark.createDataFrame(movie_rating, schema)

movie_rating_df




userId,movieId,rating
1,6,2.0
1,22,3.0
1,32,2.0
1,50,5.0
1,110,4.0
1,164,3.0
1,198,3.0
1,260,5.0
1,296,4.0
1,303,3.0


In [15]:
# description of created schema 
movie_rating_df.printSchema()

root
 |-- userId: long (nullable = true)
 |-- movieId: long (nullable = true)
 |-- rating: double (nullable = true)



In [16]:
#splitting dataset to train the model as 80% for train and remaining for test data.
(train, test) = movie_rating_df.randomSplit([0.8, 0.2], seed=87)

In [17]:
# 1st Recommender model - Alternating Least Square (ALS) Matrix Factorization in Collaborative Filtering on rating (as actual values) 

als = ALS(rank=10, maxIter=15, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

model = als.fit(train)

pred = model.transform(test)

pred = pred.selectExpr("userId as userId","movieId as movieId","rating as rating","prediction as implicit")

pred.show(truncate=False)

24/01/23 12:01:51 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/01/23 12:01:51 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/01/23 12:01:51 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/01/23 12:01:51 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


+------+-------+------+---------+
|userId|movieId|rating|implicit |
+------+-------+------+---------+
|235   |148    |5.0   |2.008149 |
|511   |148    |3.0   |1.3598946|
|31    |471    |4.5   |2.1048026|
|159   |471    |4.0   |4.2238326|
|512   |471    |1.0   |2.8343349|
|348   |471    |4.0   |5.82608  |
|40    |471    |4.0   |2.7932863|
|507   |471    |5.0   |3.3056219|
|484   |471    |2.0   |3.6099942|
|695   |471    |4.0   |3.4212964|
|381   |471    |4.0   |3.42594  |
|114   |471    |4.0   |3.698639 |
|100   |471    |4.0   |3.2666426|
|555   |471    |3.5   |4.31936  |
|7     |471    |4.0   |4.329463 |
|312   |471    |3.0   |3.8166485|
|364   |471    |4.0   |3.1999168|
|526   |471    |3.0   |5.187223 |
|284   |471    |1.0   |2.888351 |
|153   |471    |3.0   |2.7273307|
+------+-------+------+---------+
only showing top 20 rows



In [18]:
#calculating RMSE and MAE to evaluate performance of the models. 

eval_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="implicit")
eval_mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="implicit")


rmse = eval_rmse.evaluate(pred)
mae = eval_mae.evaluate(pred)


print("RMSE of ALS:", rmse)
print("MAE of ALS:", mae)




RMSE of ALS: 1.2020461630781316
MAE of ALS: 0.8919637477014059


In [19]:
# 2nd Recommender model - Alternating Least Square (ALS) Matrix Factorization in Collaborative Filtering on designed implicit feedback values 


(train_implicit, test_implicit) = pred.randomSplit([0.8, 0.2], seed=87)

als_implicit = ALS(rank=10, maxIter=15, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="implicit", coldStartStrategy="drop")

model_implicit = als_implicit.fit(train_implicit)

pred_implicit = model_implicit.transform(test_implicit)

pred_implicit.show(truncate=False)

+------+-------+------+---------+----------+
|userId|movieId|rating|implicit |prediction|
+------+-------+------+---------+----------+
|235   |148    |5.0   |2.008149 |0.90445346|
|159   |471    |4.0   |4.2238326|3.2099798 |
|512   |471    |1.0   |2.8343349|2.2843814 |
|381   |471    |4.0   |3.42594  |2.666472  |
|114   |471    |4.0   |3.698639 |2.5128846 |
|268   |833    |5.0   |1.8988763|1.3751497 |
|516   |1088   |4.0   |3.3173833|3.7465203 |
|310   |1088   |2.5   |3.1706061|3.360428  |
|396   |1342   |2.0   |3.460894 |2.152485  |
|255   |1580   |4.0   |3.6053696|1.8540831 |
|48    |1580   |5.0   |4.3630586|2.6397533 |
|455   |1580   |4.0   |2.8057783|3.1366482 |
|37    |1580   |4.5   |3.9524019|3.4095469 |
|510   |1580   |5.0   |2.8464563|1.6984986 |
|371   |1580   |4.5   |4.9224043|1.7642393 |
|228   |1580   |4.0   |3.623867 |0.6637176 |
|611   |1580   |1.0   |2.75494  |5.258436  |
|570   |1591   |3.0   |3.2005563|4.3265305 |
|332   |1645   |3.0   |3.781349 |3.8138096 |
|8     |16

In [20]:
eval_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
eval_mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")


rmse = eval_rmse.evaluate(pred_implicit)
mae = eval_mae.evaluate(pred_implicit)


print("RMSE of ALS_Implicit Feedback:", rmse)
print("MAE of ALS_Implicit Feedback:", mae)

RMSE of ALS_Implicit Feedback: 2.0638107307078846
MAE of ALS_Implicit Feedback: 1.6041012287458023


When compared these two models, the 1st model ( ALS on rating ) shows better performance compared to 2nd model (ALS on implicit feedback) according to error metrics such as Root Mean Square Error (RMSE) and Mean Absolute Error (MAE). 

## Task – 3 Text Analysis

In [21]:
spark = SparkSession.builder.appName("textAnalysis").getOrCreate()

In [22]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf


In [23]:
# download the data in Dataset folder.



!curl -o Datasets/aclImdb_v1.tar.gz https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  1229k      0  0:01:06  0:01:06 --:--:-- 1222k15.3M    0     0   878k      0  0:01:33  0:00:17  0:01:16  966k


In [24]:
#Extract the tar file as dataset

import tarfile

def tarfile_extract(tar_file, output_dir=os.getcwd()+'/Datasets'):
    tar = tarfile.open(tar_file, 'r:gz')
    total_files = sum(1 for _ in tar)
    tar.extractall(output_dir, members=extract_progress(tar, total_files))
    tar.close()

def extract_progress(tar, total_files):
    for member in tar:
        yield member
        total_files -= 1
        print(f"Remaining files: {total_files}", end='\r')
    print("\nExtraction completed.")

wd = os.getcwd()
tarfile_extract(wd+'/Datasets/aclImdb_v1.tar.gz')


Remaining files: 000000
Extraction completed.


In [25]:

#using 'alldata' list to store all the files in the directories
alldata=[]


#collecting data in train/pos folder
for fname in os.listdir(wd+'/Datasets/aclImdb/train/pos'):
    with open(os.path.join(wd+'/Datasets/aclImdb/train/pos', fname), encoding = 'utf-8') as infile:
        for line in infile:
            alldata.append((line,'train','pos'))

#collecting data in train/neg folder
for fname in os.listdir(wd+'/Datasets/aclImdb/train/neg'):
    with open(os.path.join(wd+'/Datasets/aclImdb/train/neg', fname), encoding = 'utf-8') as infile:
        for line in infile:
            alldata.append((line,'train','neg'))
#collecting data in test/pos folder
for fname in os.listdir(wd+'/Datasets/aclImdb/test/pos'):
    with open(os.path.join(wd+'/Datasets/aclImdb/test/pos', fname), encoding = 'utf-8') as infile:
        for line in infile:
            alldata.append((line,'test','pos'))
#collecting data in test/neg folder
for fname in os.listdir(wd+'/Datasets/aclImdb/test/neg'):
    with open(os.path.join(wd+'/Datasets/aclImdb/test/neg', fname), encoding = 'utf-8') as infile:
        for line in infile:
            alldata.append((line,'test','neg'))




In [26]:

from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType

appName = "list to Spark Data Frame"
master = "local"

# Create Spark session
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .getOrCreate()

# List
data = alldata

# Create a schema for the dataframe
schema = StructType([
    StructField('content', StringType(), True),
    StructField('label', StringType(), True),
    StructField('sentiemtn', StringType(), True)
])

# Convert list to RDD
rdd = spark.sparkContext.parallelize(data)

# Create data frame
df = spark.createDataFrame(rdd,schema)
print(df.schema)
df.show()

StructType(List(StructField(content,StringType,true),StructField(label,StringType,true),StructField(sentiemtn,StringType,true)))
+--------------------+-----+---------+
|             content|label|sentiemtn|
+--------------------+-----+---------+
|First of all this...|train|      pos|
|Its a good thing ...|train|      pos|
|Hard to categoriz...|train|      pos|
|Recently when i w...|train|      pos|
|First, let me say...|train|      pos|
|OK heres what I s...|train|      pos|
|The real story (t...|train|      pos|
|Since I am so int...|train|      pos|
|The Ruth Snyder -...|train|      pos|
|49. PAPERHOUSE (t...|train|      pos|
|The Woman in Blac...|train|      pos|
|Part II or former...|train|      pos|
|After watching tw...|train|      pos|
|I really like Ric...|train|      pos|
|I began watching ...|train|      pos|
|Holes, originally...|train|      pos|
|After being force...|train|      pos|
|Radiofreccia is a...|train|      pos|
|I loved October S...|train|      pos|
|This is a qu

24/01/23 12:04:52 WARN TaskSetManager: Stage 705 contains a task of very large size (16318 KiB). The maximum recommended task size is 1000 KiB.


In [27]:
df.count()

24/01/23 12:04:52 WARN TaskSetManager: Stage 706 contains a task of very large size (16318 KiB). The maximum recommended task size is 1000 KiB.


50000

In [28]:
# Schema of created Spark Dataframe

df.printSchema()

root
 |-- content: string (nullable = true)
 |-- label: string (nullable = true)
 |-- sentiemtn: string (nullable = true)



###Tokenization

In [29]:
#using RegexTokenizer for tokenizing contents

tokenizer = RegexTokenizer(inputCol="content", outputCol="tokenized_content", pattern="\\W") # used regexp to determine pattern as 'not word'

countTokens = udf(lambda w: len(w), IntegerType())

tokenized = tokenizer.transform(df)

tokenized.show()

24/01/23 12:04:54 WARN TaskSetManager: Stage 708 contains a task of very large size (16318 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+---------+--------------------+
|             content|label|sentiemtn|   tokenized_content|
+--------------------+-----+---------+--------------------+
|First of all this...|train|      pos|[first, of, all, ...|
|Its a good thing ...|train|      pos|[its, a, good, th...|
|Hard to categoriz...|train|      pos|[hard, to, catego...|
|Recently when i w...|train|      pos|[recently, when, ...|
|First, let me say...|train|      pos|[first, let, me, ...|
|OK heres what I s...|train|      pos|[ok, heres, what,...|
|The real story (t...|train|      pos|[the, real, story...|
|Since I am so int...|train|      pos|[since, i, am, so...|
|The Ruth Snyder -...|train|      pos|[the, ruth, snyde...|
|49. PAPERHOUSE (t...|train|      pos|[49, paperhouse, ...|
|The Woman in Blac...|train|      pos|[the, woman, in, ...|
|Part II or former...|train|      pos|[part, ii, or, fo...|
|After watching tw...|train|      pos|[after, watching,...|
|I really like Ric...|train|      pos|[i

### Removing Stop Words

In [30]:
type(tokenized)

pyspark.sql.dataframe.DataFrame

In [31]:
from pyspark.ml.feature import StopWordsRemover

tokenized.show()

+--------------------+-----+---------+--------------------+
|             content|label|sentiemtn|   tokenized_content|
+--------------------+-----+---------+--------------------+
|First of all this...|train|      pos|[first, of, all, ...|
|Its a good thing ...|train|      pos|[its, a, good, th...|
|Hard to categoriz...|train|      pos|[hard, to, catego...|
|Recently when i w...|train|      pos|[recently, when, ...|
|First, let me say...|train|      pos|[first, let, me, ...|
|OK heres what I s...|train|      pos|[ok, heres, what,...|
|The real story (t...|train|      pos|[the, real, story...|
|Since I am so int...|train|      pos|[since, i, am, so...|
|The Ruth Snyder -...|train|      pos|[the, ruth, snyde...|
|49. PAPERHOUSE (t...|train|      pos|[49, paperhouse, ...|
|The Woman in Blac...|train|      pos|[the, woman, in, ...|
|Part II or former...|train|      pos|[part, ii, or, fo...|
|After watching tw...|train|      pos|[after, watching,...|
|I really like Ric...|train|      pos|[i

24/01/23 12:04:54 WARN TaskSetManager: Stage 709 contains a task of very large size (16318 KiB). The maximum recommended task size is 1000 KiB.


In [32]:
df_tokenized = tokenized.select("tokenized_content").withColumn("tokenCount", countTokens(col("tokenized_content")))

In [33]:
type(df_tokenized)

pyspark.sql.dataframe.DataFrame

In [34]:
# SWR -> stop word remover
SWR  = StopWordsRemover (inputCol='tokenized_content', outputCol='SWRed')


#See the result of removal operation
SWR.transform(df_tokenized).select('SWRed').show(truncate=False)

24/01/23 12:04:56 WARN TaskSetManager: Stage 710 contains a task of very large size (16318 KiB). The maximum recommended task size is 1000 KiB.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------